# Group 5 - Module 7: Dialogue systems and question answering

***
### Group Members:
* **Nils Dunlop, 20010127-2359, Applied Data Science, e-mail: gusdunlni@student.gu.se (16 hours)**
* **Francisco Erazo, 19930613-9214, Applied Data Science, e-mail: guserafr@student.gu.se (16 hours)**

#### **We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions." (This is independent and additional to any declaration that you may encounter in the electronic submission system.)**

# Assignment 7
***

## Problem 1: Reading and Summary
***


## Problem 2: Implement A Simple Dialogue System
***

## References
***

- Choudhary, A. (2018). Reinforcement Learning Guide: Solving the Multi-Armed Bandit Problem from Scratch in Python. [online] Analytics Vidhya. Available at: https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/ [Accessed 23 Feb. 2024].

- Nested Software (2019). Tic-Tac-Toe with MCTS. [online] DEV Community. Available at: https://dev.to/nestedsoftware/tic-tac-toe-with-mcts-2h5k [Accessed 26 Feb. 2024].

- GfG (2019). ML Monte Carlo Tree Search (MCTS). [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/ [Accessed 26 Feb. 2024].

## Self Check
***
- Have you answered all questions to the best of your ability?
Yes, we have.
- Is all the required information on the front page, is the file name correct etc.?
Indeed, all the required information on the front page has been included.
- Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?)
We have checked, and everything looks good.